In [ ]:
import yolov7
import cv2
import torch
from datetime import datetime
import numpy as np

In [ ]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [ ]:
CAR_HEIGHT = 1600 #in mm
BIKE_HEIGHT = 1450 #in mm
TRUCK_HEIGHT = 3500 #in mm
FOCAL_LENGTH = 3.67 # in mm
SENSOR_HEIGHT = 3.5 # in mm 

In [ ]:
model_yolo = yolov7.load('./data_models/yolov7_best.pt', device=device)
model_yolo.conf = 0.25  # NMS confidence threshold
model_yolo.iou = 0.45  # NMS IoU threshold
model_yolo.classes = None  # (optional list) filter by class

In [ ]:
vehicle_idx_dict = {0: "car", 1: "truck", 2: "bus", 3: "motorcycle"}

In [ ]:
vidcap = cv2.VideoCapture("../emission_mapping/dataset/videos/demo_3.mp4")  ## Load video here. Provide complete path to the video here
IMAGE_WIDTH = int(vidcap.get(3))
IMAGE_HEIGHT = int(vidcap.get(4))
FPS = vidcap.get(cv2.CAP_PROP_FPS)
WRITE_VIDEO = False
DIST_THRESH = 35

ret, frame = vidcap.read()

if WRITE_VIDEO:
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.mp4", fourcc, int(FPS), (IMAGE_WIDTH, IMAGE_HEIGHT))

In [ ]:
i = 0
while ret:
    print(i, end="\r")
    ret, frame = vidcap.read()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    IMAGE_HEIGHT, IMAGE_WIDTH, _ = frame_rgb.shape
    
    results = model_yolo(frame, size = 640, augment = False) # Augment = True for TTA

    # Parse results
    predictions = results.pred[0]
    boxes = predictions[:, :4] # x1, y1, x2, y2
    scores = predictions[:, 4]
    categories = predictions[:, 5]
    
    for vehicle_idx in range(len(boxes)):
        xmin, ymin, xmax, ymax = list(map(int, boxes[vehicle_idx]))
        height_box = ymax - ymin
        width_box = xmax - xmin
        center_box = (int(width_box/2), int(height_box/2))
        vehicle_category = vehicle_idx_dict[int(categories[vehicle_idx])]
        
        if vehicle_category == "car":
            distance_car = round(np.divide((FOCAL_LENGTH * CAR_HEIGHT * IMAGE_HEIGHT)/(height_box * SENSOR_HEIGHT), 1000), 2)
        elif vehicle_category == "truck" or vehicle_category == "bus":
            distance_truck_bus = round(np.divide((FOCAL_LENGTH * TRUCK_HEIGHT * IMAGE_HEIGHT)/(height_box * SENSOR_HEIGHT), 1000), 2)
        else: 
            distance_motorcycle = round(np.divide((FOCAL_LENGTH * BIKE_HEIGHT * IMAGE_HEIGHT)/(height_box * SENSOR_HEIGHT), 1000), 2)
    i += 1